<a href="https://colab.research.google.com/github/nikhil-mathews/MastersPr_Predicting-Human-Pathogen-PPIs-using-Natural-Language-Processing-methods/blob/main/3.Attention%20%26%20Transformers/att_CNN_3D%2B4D%2B5D%2B6D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
#Google colab does not have pickle
try:
  import pickle5 as pickle
except:
  !pip install pickle5
  import pickle5 as pickle
import os
import seaborn as sns
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D,Flatten, Attention
from keras.layers import Conv1D, MaxPooling1D, Embedding,  Concatenate, Lambda,RepeatVector,Dot
from keras.models import Model
from sklearn.metrics import roc_auc_score,confusion_matrix,roc_curve, auc
from numpy import random
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model

import sys
sys.path.insert(0,'/content/drive/MyDrive/ML_Data/')
import functions as f


def load_data(D=1,randomize=False):
    try:
        with open('/content/drive/MyDrive/ML_Data/df_train_'+str(D)+'D.pickle', 'rb') as handle:
            df_train = pickle.load(handle)
    except:
        df_train = pd.read_pickle("C:/Users/nik00/py/proj/hyppi-train.pkl")
    try:
        with open('/content/drive/MyDrive/ML_Data/df_test_'+str(D)+'D.pickle', 'rb') as handle:
            df_test = pickle.load(handle)
    except:
        df_test = pd.read_pickle("C:/Users/nik00/py/proj/hyppi-independent.pkl")
    if randomize:
        return shuff_together(df_train,df_test)
    else:
        return df_train,df_test

     |████████████████████████████████| 133kB 8.3MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219272 sha256=83fe98c44a72b5cef19afbb95942043932f7d8a6278b7a46aae3c2c2c83c42bf
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [ ]:
df_train,df_test = load_data(3)
print('The data used will be:')
df_train[['Human','Yersinia']]

The data used will be:


,Human,Yersinia
0,"[MKD, KDK, DKQ, KQK, QKK, KKK, KKK, KKE, KER, ...","[MAK, AKA, KAS, ASR, SRH, RHN, HNL, NLS, LSI, ..."
1,"[MSW, SWS, WSG, SGL, GLL, LLH, LHG, HGL, GLN, ...","[MQH, QHV, HVT, VTG, TGS, GSK, SKR, KRR, RRL, ..."
2,"[MSL, SLF, LFD, FDL, DLF, LFR, FRG, RGF, GFF, ...","[MAE, AEL, ELP, LPA, PAK, AKR, KRR, RRF, RFT, ..."
3,"[MAV, AVA, VAP, APR, PRL, RLF, LFG, FGG, GGL, ...","[MRI, RIF, IFA, FAI, AIS, ISC, SCS, CSS, SSY, ..."
4,"[MST, STI, TIQ, IQS, QSE, SET, ETD, TDC, DCY, ...","[MSY, SYA, YAF, AFP, FPG, PGT, GTF, TFP, FPG, ..."
...,...,...
6265,"[MSY, SYF, YFG, FGE, GEH, EHF, HFW, FWG, WGE, ...","[MIT, ITT, TTD, TDG, DGN, GNS, NSA, SAV, AVA, ..."
6266,"[MTV, TVG, VGK, GKS, KSS, SSK, SKM, KML, MLQ, ...","[MSQ, SQP, QPP, PPF, PFW, FWQ, WQQ, QQK, QKT, ..."
6267,"[MNN, NNL, NLS, LSF, SFS, FSE, SEL, ELC, LCC, ...","[MSE, SED, EDR, DRH, RHQ, HQQ, QQR, QRQ, RQQ, ..."
6268,"[MAP, APE, PEI, EIN, INL, NLP, LPG, PGP, GPM, ...","[MKN, KNL, NLS, LSF, SFV, FVA, VAG, AGL, GLV, ..."


In [ ]:
data1,data2,data1_test,data2_test,num_words,MAX_SEQUENCE_LENGTH,MAX_VOCAB_SIZE = f.get_seq_data_doubleip(7000,1000,df_train,df_test, pad='pre')

MAX_VOCAB_SIZE is 7000
MAX_SEQUENCE_LENGTH is 1000
max sequences1_train length: 8624
min sequences1_train length: 37
median sequences1_train length: 467
max word index sequences1_train: 6999
max sequences2_train length: 3704
min sequences2_train length: 29
median sequences2_train length: 329
max word index sequences2_train: 6999
Found 8000 unique tokens in tokenizer1.
Found 7998 unique tokens in tokenizer2.
pre padding
Shape of data1 tensor: (6270, 1000)
Shape of data2 tensor: (6270, 1000)
max test_sequences1 length: 5530
min test_sequences1 length: 37
median test_sequences1 length: 484
max test_sequences2 length: 3704
min test_sequences2 length: 29
median test_sequences2 length: 324
pre padding for test seq.
Shape of test_data1 tensor: (1514, 1000)
Shape of test_data2 tensor: (1514, 1000)
num_words is 7000


In [ ]:
EMBEDDING_DIM = 5
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 20
M_1D=10
DROP = 0.6

ip1 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP)
ip2 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP)

concatenator = Concatenate(axis=1)
x = concatenator([ip1.output, ip2.output])
x = Dense(128)(x)
#x = concatenator([x1.output, x2.output]) # output is N x 4M
#x = Dense(128, activation='relu')(x)
x = Dropout(DROP)(x)
output = Dense(1, activation="sigmoid",name="Final")(x)
model_doubleip = Model(inputs=[ip1.input,ip2.input], outputs=output)
#plot_model(model1D_CNN_doubleip, to_file='model_plot.png', show_shapes=True, show_layer_names=False)
model_doubleip.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
trains = [data1,data2,data2,data1]
tests = [data1_test,data2_test,data2_test,data1_test]

model_doubleip.fit(trains, df_train['label'].values,batch_size=BATCH_SIZE, epochs=EPOCHS,validation_data=(tests,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model_doubleip.predict(tests)))

Epoch 1/20
49/49 [==============================] - 9s 140ms/step - loss: 0.6651 - accuracy: 0.5956 - val_loss: 0.6089 - val_accuracy: 0.6651
Epoch 2/20
49/49 [==============================] - 7s 135ms/step - loss: 0.6263 - accuracy: 0.6497 - val_loss: 0.5908 - val_accuracy: 0.6889
Epoch 3/20
49/49 [==============================] - 7s 135ms/step - loss: 0.6117 - accuracy: 0.6628 - val_loss: 0.5816 - val_accuracy: 0.6968
Epoch 4/20
49/49 [==============================] - 7s 136ms/step - loss: 0.5868 - accuracy: 0.6814 - val_loss: 0.5635 - val_accuracy: 0.7094
Epoch 5/20
49/49 [==============================] - 7s 135ms/step - loss: 0.5658 - accuracy: 0.7033 - val_loss: 0.5451 - val_accuracy: 0.7180
Epoch 6/20
49/49 [==============================] - 7s 134ms/step - loss: 0.5429 - accuracy: 0.7169 - val_loss: 0.5368 - val_accuracy: 0.7186
Epoch 7/20
49/49 [==============================] - 7s 134ms/step - loss: 0.5072 - accuracy: 0.7495 - val_loss: 0.5197 - val_accuracy: 0.7338
Epoch 

In [ ]:
df_train,df_test = load_data(4)
print('The data used will be:')
df_train[['Human','Yersinia']]

The data used will be:


,Human,Yersinia
0,"[MKDK, KDKQ, DKQK, KQKK, QKKK, KKKK, KKKE, KKE...","[MAKA, AKAS, KASR, ASRH, SRHN, RHNL, HNLS, NLS..."
1,"[MSWS, SWSG, WSGL, SGLL, GLLH, LLHG, LHGL, HGL...","[MQHV, QHVT, HVTG, VTGS, TGSK, GSKR, SKRR, KRR..."
2,"[MSLF, SLFD, LFDL, FDLF, DLFR, LFRG, FRGF, RGF...","[MAEL, AELP, ELPA, LPAK, PAKR, AKRR, KRRF, RRF..."
3,"[MAVA, AVAP, VAPR, APRL, PRLF, RLFG, LFGG, FGG...","[MRIF, RIFA, IFAI, FAIS, AISC, ISCS, SCSS, CSS..."
4,"[MSTI, STIQ, TIQS, IQSE, QSET, SETD, ETDC, TDC...","[MSYA, SYAF, YAFP, AFPG, FPGT, PGTF, GTFP, TFP..."
...,...,...
6265,"[MSYF, SYFG, YFGE, FGEH, GEHF, EHFW, HFWG, FWG...","[MITT, ITTD, TTDG, TDGN, DGNS, GNSA, NSAV, SAV..."
6266,"[MTVG, TVGK, VGKS, GKSS, KSSK, SSKM, SKML, KML...","[MSQP, SQPP, QPPF, PPFW, PFWQ, FWQQ, WQQK, QQK..."
6267,"[MNNL, NNLS, NLSF, LSFS, SFSE, FSEL, SELC, ELC...","[MSED, SEDR, EDRH, DRHQ, RHQQ, HQQR, QQRQ, QRQ..."
6268,"[MAPE, APEI, PEIN, EINL, INLP, NLPG, LPGP, PGP...","[MKNL, KNLS, NLSF, LSFV, SFVA, FVAG, VAGL, AGL..."


In [ ]:
 data1,data2,data1_test,data2_test,num_words,MAX_SEQUENCE_LENGTH,MAX_VOCAB_SIZE = f.get_seq_data_doubleip(100000,1000,df_train,df_test,pad = 'pre')

MAX_VOCAB_SIZE is 100000
MAX_SEQUENCE_LENGTH is 1000
max sequences1_train length: 8063
min sequences1_train length: 34
median sequences1_train length: 440
max word index sequences1_train: 99999
max sequences2_train length: 3707
min sequences2_train length: 26
median sequences2_train length: 320
max word index sequences2_train: 99999
Found 155225 unique tokens in tokenizer1.
Found 134906 unique tokens in tokenizer2.
pre padding
Shape of data1 tensor: (6270, 1000)
Shape of data2 tensor: (6270, 1000)
max test_sequences1 length: 5224
min test_sequences1 length: 34
median test_sequences1 length: 449
max test_sequences2 length: 3707
min test_sequences2 length: 24
median test_sequences2 length: 308
pre padding for test seq.
Shape of test_data1 tensor: (1514, 1000)
Shape of test_data2 tensor: (1514, 1000)
num_words is 100000


In [ ]:
EMBEDDING_DIM = 10
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10
M_1D=10
DROP = 0.7

ip1 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP)
ip2 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP)

concatenator = Concatenate(axis=1)
x = concatenator([ip1.output, ip2.output])
x = Dense(128)(x)
#x = concatenator([x1.output, x2.output]) # output is N x 4M
#x = Dense(128, activation='relu')(x)
x = Dropout(DROP)(x)
output = Dense(1, activation="sigmoid",name="Final")(x)
model_doubleip = Model(inputs=[ip1.input,ip2.input], outputs=output)
#plot_model(model1D_CNN_doubleip, to_file='model_plot.png', show_shapes=True, show_layer_names=False)
model_doubleip.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
trains = [data1,data2,data2,data1]
tests = [data1_test,data2_test,data2_test,data1_test]

model_doubleip.fit(trains, df_train['label'].values,batch_size=BATCH_SIZE, epochs=EPOCHS,validation_data=(tests,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model_doubleip.predict(tests)))

Epoch 1/10
49/49 [==============================] - 11s 194ms/step - loss: 0.6891 - accuracy: 0.5708 - val_loss: 0.6066 - val_accuracy: 0.6750
Epoch 2/10
49/49 [==============================] - 9s 187ms/step - loss: 0.6291 - accuracy: 0.6538 - val_loss: 0.6018 - val_accuracy: 0.6783
Epoch 3/10
49/49 [==============================] - 9s 188ms/step - loss: 0.5981 - accuracy: 0.6796 - val_loss: 0.5618 - val_accuracy: 0.7133
Epoch 4/10
49/49 [==============================] - 9s 186ms/step - loss: 0.5653 - accuracy: 0.7242 - val_loss: 0.5172 - val_accuracy: 0.7457
Epoch 5/10
49/49 [==============================] - 9s 188ms/step - loss: 0.4809 - accuracy: 0.7845 - val_loss: 0.4734 - val_accuracy: 0.7787
Epoch 6/10
49/49 [==============================] - 9s 186ms/step - loss: 0.3568 - accuracy: 0.8482 - val_loss: 0.4481 - val_accuracy: 0.8025
Epoch 7/10
49/49 [==============================] - 9s 187ms/step - loss: 0.2568 - accuracy: 0.8912 - val_loss: 0.5789 - val_accuracy: 0.7814
Epoch

In [ ]:
df_train,df_test = load_data(5)
print('The data used will be:')
df_train[['Human','Yersinia']]

The data used will be:


,Human,Yersinia
0,"[MKDKQ, KDKQK, DKQKK, KQKKK, QKKKK, KKKKE, KKK...","[MAKAS, AKASR, KASRH, ASRHN, SRHNL, RHNLS, HNL..."
1,"[MSWSG, SWSGL, WSGLL, SGLLH, GLLHG, LLHGL, LHG...","[MQHVT, QHVTG, HVTGS, VTGSK, TGSKR, GSKRR, SKR..."
2,"[MSLFD, SLFDL, LFDLF, FDLFR, DLFRG, LFRGF, FRG...","[MAELP, AELPA, ELPAK, LPAKR, PAKRR, AKRRF, KRR..."
3,"[MAVAP, AVAPR, VAPRL, APRLF, PRLFG, RLFGG, LFG...","[MRIFA, RIFAI, IFAIS, FAISC, AISCS, ISCSS, SCS..."
4,"[MSTIQ, STIQS, TIQSE, IQSET, QSETD, SETDC, ETD...","[MSYAF, SYAFP, YAFPG, AFPGT, FPGTF, PGTFP, GTF..."
...,...,...
6265,"[MSYFG, SYFGE, YFGEH, FGEHF, GEHFW, EHFWG, HFW...","[MITTD, ITTDG, TTDGN, TDGNS, DGNSA, GNSAV, NSA..."
6266,"[MTVGK, TVGKS, VGKSS, GKSSK, KSSKM, SSKML, SKM...","[MSQPP, SQPPF, QPPFW, PPFWQ, PFWQQ, FWQQK, WQQ..."
6267,"[MNNLS, NNLSF, NLSFS, LSFSE, SFSEL, FSELC, SEL...","[MSEDR, SEDRH, EDRHQ, DRHQQ, RHQQR, HQQRQ, QQR..."
6268,"[MAPEI, APEIN, PEINL, EINLP, INLPG, NLPGP, LPG...","[MKNLS, KNLSF, NLSFV, LSFVA, SFVAG, FVAGL, VAG..."


In [ ]:
 data1,data2,data1_test,data2_test,num_words,MAX_SEQUENCE_LENGTH,MAX_VOCAB_SIZE = f.get_seq_data_doubleip(500000,1000,df_train,df_test,pad = 'pre')

MAX_VOCAB_SIZE is 500000
MAX_SEQUENCE_LENGTH is 1000
max sequences1_train length: 5301
min sequences1_train length: 12
median sequences1_train length: 327
max word index sequences1_train: 499999
max sequences2_train length: 3706
min sequences2_train length: 9
median sequences2_train length: 307
max word index sequences2_train: 499999
Found 1330540 unique tokens in tokenizer1.
Found 639136 unique tokens in tokenizer2.
pre padding
Shape of data1 tensor: (6270, 1000)
Shape of data2 tensor: (6270, 1000)
max test_sequences1 length: 4674
min test_sequences1 length: 10
median test_sequences1 length: 271
max test_sequences2 length: 3706
min test_sequences2 length: 5
median test_sequences2 length: 244
pre padding for test seq.
Shape of test_data1 tensor: (1514, 1000)
Shape of test_data2 tensor: (1514, 1000)
num_words is 500000


In [ ]:
EMBEDDING_DIM = 15
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10
M_1D=10
DROP = 0.8

ip1 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP)
ip2 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP)

concatenator = Concatenate(axis=1)
x = concatenator([ip1.output, ip2.output])
x = Dense(128)(x)
#x = concatenator([x1.output, x2.output]) # output is N x 4M
#x = Dense(128, activation='relu')(x)
x = Dropout(DROP)(x)
output = Dense(1, activation="sigmoid",name="Final")(x)
model_doubleip = Model(inputs=[ip1.input,ip2.input], outputs=output)
#plot_model(model1D_CNN_doubleip, to_file='model_plot.png', show_shapes=True, show_layer_names=False)
model_doubleip.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
trains = [data1,data2,data2,data1]
tests = [data1_test,data2_test,data2_test,data1_test]

model_doubleip.fit(trains, df_train['label'].values,batch_size=BATCH_SIZE, epochs=EPOCHS,validation_data=(tests,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model_doubleip.predict(tests)))

Epoch 1/10
49/49 [==============================] - 21s 380ms/step - loss: 0.7634 - accuracy: 0.5560 - val_loss: 0.6109 - val_accuracy: 0.6374
Epoch 2/10
49/49 [==============================] - 18s 371ms/step - loss: 0.6032 - accuracy: 0.6875 - val_loss: 0.5980 - val_accuracy: 0.6863
Epoch 3/10
49/49 [==============================] - 18s 374ms/step - loss: 0.5850 - accuracy: 0.7035 - val_loss: 0.5966 - val_accuracy: 0.6625
Epoch 4/10
49/49 [==============================] - 18s 372ms/step - loss: 0.5568 - accuracy: 0.7032 - val_loss: 0.5628 - val_accuracy: 0.7318
Epoch 5/10
49/49 [==============================] - 18s 368ms/step - loss: 0.5116 - accuracy: 0.7523 - val_loss: 0.5158 - val_accuracy: 0.7748
Epoch 6/10
49/49 [==============================] - 18s 375ms/step - loss: 0.4059 - accuracy: 0.8288 - val_loss: 0.4616 - val_accuracy: 0.8091
Epoch 7/10
49/49 [==============================] - 18s 370ms/step - loss: 0.3147 - accuracy: 0.8720 - val_loss: 0.5306 - val_accuracy: 0.7985

In [ ]:
df_train,df_test = load_data(6)
print('The data used will be:')
df_train[['Human','Yersinia']]

The data used will be:


,Human,Yersinia
0,"[MKDKQK, KDKQKK, DKQKKK, KQKKKK, QKKKKE, KKKKE...","[MAKASR, AKASRH, KASRHN, ASRHNL, SRHNLS, RHNLS..."
1,"[MSWSGL, SWSGLL, WSGLLH, SGLLHG, GLLHGL, LLHGL...","[MQHVTG, QHVTGS, HVTGSK, VTGSKR, TGSKRR, GSKRR..."
2,"[MSLFDL, SLFDLF, LFDLFR, FDLFRG, DLFRGF, LFRGF...","[MAELPA, AELPAK, ELPAKR, LPAKRR, PAKRRF, AKRRF..."
3,"[MAVAPR, AVAPRL, VAPRLF, APRLFG, PRLFGG, RLFGG...","[MRIFAI, RIFAIS, IFAISC, FAISCS, AISCSS, ISCSS..."
4,"[MSTIQS, STIQSE, TIQSET, IQSETD, QSETDC, SETDC...","[MSYAFP, SYAFPG, YAFPGT, AFPGTF, FPGTFP, PGTFP..."
...,...,...
6265,"[MSYFGE, SYFGEH, YFGEHF, FGEHFW, GEHFWG, EHFWG...","[MITTDG, ITTDGN, TTDGNS, TDGNSA, DGNSAV, GNSAV..."
6266,"[MTVGKS, TVGKSS, VGKSSK, GKSSKM, KSSKML, SSKML...","[MSQPPF, SQPPFW, QPPFWQ, PPFWQQ, PFWQQK, FWQQK..."
6267,"[MNNLSF, NNLSFS, NLSFSE, LSFSEL, SFSELC, FSELC...","[MSEDRH, SEDRHQ, EDRHQQ, DRHQQR, RHQQRQ, HQQRQ..."
6268,"[MAPEIN, APEINL, PEINLP, EINLPG, INLPGP, NLPGP...","[MKNLSF, KNLSFV, NLSFVA, LSFVAG, SFVAGL, FVAGL..."


In [ ]:
 data1,data2,data1_test,data2_test,num_words,MAX_SEQUENCE_LENGTH,MAX_VOCAB_SIZE = f.get_seq_data_doubleip(850000,1000,df_train,df_test,pad = 'pre')

MAX_VOCAB_SIZE is 850000
MAX_SEQUENCE_LENGTH is 1000
max sequences1_train length: 5630
min sequences1_train length: 0
median sequences1_train length: 199
max word index sequences1_train: 849999
max sequences2_train length: 3705
min sequences2_train length: 0
median sequences2_train length: 329
max word index sequences2_train: 849999
Found 2438322 unique tokens in tokenizer1.
Found 864366 unique tokens in tokenizer2.
pre padding
Shape of data1 tensor: (6270, 1000)
Shape of data2 tensor: (6270, 1000)
max test_sequences1 length: 5630
min test_sequences1 length: 0
median test_sequences1 length: 70
max test_sequences2 length: 3705
min test_sequences2 length: 0
median test_sequences2 length: 284
pre padding for test seq.
Shape of test_data1 tensor: (1514, 1000)
Shape of test_data2 tensor: (1514, 1000)
num_words is 850000


In [ ]:
EMBEDDING_DIM = 15
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10
M_1D=10
DROP = 0.8

ip1 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP)
ip2 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP)

concatenator = Concatenate(axis=1)
x = concatenator([ip1.output, ip2.output])
x = Dense(128)(x)
#x = concatenator([x1.output, x2.output]) # output is N x 4M
#x = Dense(128, activation='relu')(x)
x = Dropout(DROP)(x)
output = Dense(1, activation="sigmoid",name="Final")(x)
model_doubleip = Model(inputs=[ip1.input,ip2.input], outputs=output)
#plot_model(model1D_CNN_doubleip, to_file='model_plot.png', show_shapes=True, show_layer_names=False)
model_doubleip.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
trains = [data1,data2,data2,data1]
tests = [data1_test,data2_test,data2_test,data1_test]

model_doubleip.fit(trains, df_train['label'].values,batch_size=BATCH_SIZE, epochs=EPOCHS,validation_data=(tests,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model_doubleip.predict(tests)))

Epoch 1/10
49/49 [==============================] - 30s 583ms/step - loss: 0.7745 - accuracy: 0.5410 - val_loss: 0.5858 - val_accuracy: 0.7127
Epoch 2/10
49/49 [==============================] - 28s 577ms/step - loss: 0.6023 - accuracy: 0.6970 - val_loss: 0.5743 - val_accuracy: 0.7147
Epoch 3/10
49/49 [==============================] - 28s 576ms/step - loss: 0.5618 - accuracy: 0.7210 - val_loss: 0.5592 - val_accuracy: 0.7001
Epoch 4/10
49/49 [==============================] - 28s 578ms/step - loss: 0.5259 - accuracy: 0.7428 - val_loss: 0.6001 - val_accuracy: 0.6546
Epoch 5/10
49/49 [==============================] - 28s 579ms/step - loss: 0.4505 - accuracy: 0.7969 - val_loss: 0.5512 - val_accuracy: 0.7450
Epoch 6/10
49/49 [==============================] - 28s 576ms/step - loss: 0.3661 - accuracy: 0.8502 - val_loss: 0.6475 - val_accuracy: 0.7483
Epoch 7/10
49/49 [==============================] - 28s 574ms/step - loss: 0.2659 - accuracy: 0.8862 - val_loss: 0.7011 - val_accuracy: 0.7734

#### We will try for 5D with 3X

In [ ]:
data1_5D_doubleip_pre,data2_5D_doubleip_pre,data1_test_5D_doubleip_pre,data2_test_5D_doubleip_pre,num_words_5D,MAX_SEQUENCE_LENGTH_5D,MAX_VOCAB_SIZE_5D = f.get_seq_data_doubleip(500000,1000,df_train,df_test,pad = 'pre')
data1_5D_doubleip_center,data2_5D_doubleip_center,data1_test_5D_doubleip_center,data2_test_5D_doubleip_center,num_words_5D,MAX_SEQUENCE_LENGTH_5D,MAX_VOCAB_SIZE_5D = f.get_seq_data_doubleip(500000,1000,df_train,df_test)
data1_5D_doubleip_post,data2_5D_doubleip_post,data1_test_5D_doubleip_post,data2_test_5D_doubleip_post,num_words_5D,MAX_SEQUENCE_LENGTH_5D,MAX_VOCAB_SIZE_5D = f.get_seq_data_doubleip(500000,1000,df_train,df_test,pad = 'post')

MAX_VOCAB_SIZE is 500000
MAX_SEQUENCE_LENGTH is 1000
max sequences1_train length: 5301
min sequences1_train length: 12
median sequences1_train length: 327
max word index sequences1_train: 499999
max sequences2_train length: 3706
min sequences2_train length: 9
median sequences2_train length: 307
max word index sequences2_train: 499999
Found 1330540 unique tokens in tokenizer1.
Found 639136 unique tokens in tokenizer2.
pre padding
Shape of data1 tensor: (6270, 1000)
Shape of data2 tensor: (6270, 1000)
max test_sequences1 length: 4674
min test_sequences1 length: 10
median test_sequences1 length: 271
max test_sequences2 length: 3706
min test_sequences2 length: 5
median test_sequences2 length: 244
pre padding for test seq.
Shape of test_data1 tensor: (1514, 1000)
Shape of test_data2 tensor: (1514, 1000)
num_words is 500000
MAX_VOCAB_SIZE is 500000
MAX_SEQUENCE_LENGTH is 1000
max sequences1_train length: 5301
min sequences1_train length: 12
median sequences1_train length: 327
max word index 

In [ ]:


EMBEDDING_DIM_5D = 15
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10
DROP=0.8

x1 = f.att_model(MAX_SEQUENCE_LENGTH_5D,EMBEDDING_DIM_5D,num_words_5D,DROP)
x2 = f.att_model(MAX_SEQUENCE_LENGTH_5D,EMBEDDING_DIM_5D,num_words_5D,DROP)
x3 = f.att_model(MAX_SEQUENCE_LENGTH_5D,EMBEDDING_DIM_5D,num_words_5D,DROP)
x4 = f.att_model(MAX_SEQUENCE_LENGTH_5D,EMBEDDING_DIM_5D,num_words_5D,DROP)
x5 = f.att_model(MAX_SEQUENCE_LENGTH_5D,EMBEDDING_DIM_5D,num_words_5D,DROP)
x6 = f.att_model(MAX_SEQUENCE_LENGTH_5D,EMBEDDING_DIM_5D,num_words_5D,DROP)

concatenator = Concatenate(axis=1)
x = concatenator([x1.output, x2.output, x3.output, x4.output, x5.output, x6.output])
x = Dense(128)(x)
x = Dropout(DROP)(x)
output = Dense(1, activation="sigmoid",name="Final")(x)
model5D_CNN_doubleip = Model(inputs=[x1.input, x2.input, x3.input, x4.input, x5.input, x6.input], outputs=output)

model5D_CNN_doubleip.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#plot_model(model5D_CNN_doubleip, to_file='model_plot.png', show_shapes=True, show_layer_names=False)

trains = [data1_5D_doubleip_pre,data2_5D_doubleip_pre,data2_5D_doubleip_pre,data1_5D_doubleip_pre,data1_5D_doubleip_center,data2_5D_doubleip_center,data2_5D_doubleip_center,data1_5D_doubleip_center,data1_5D_doubleip_post,data2_5D_doubleip_post,data2_5D_doubleip_post,data1_5D_doubleip_post]
tests = [data1_test_5D_doubleip_pre,data2_test_5D_doubleip_pre,data2_test_5D_doubleip_pre,data1_test_5D_doubleip_pre,data1_test_5D_doubleip_center,data2_test_5D_doubleip_center,data2_test_5D_doubleip_center,data1_test_5D_doubleip_center,data1_test_5D_doubleip_post,data2_test_5D_doubleip_post,data2_test_5D_doubleip_post,data1_test_5D_doubleip_post]


model5D_CNN_doubleip.fit(trains, df_train['label'].values, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(tests, df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model5D_CNN_doubleip.predict(tests)))


Epoch 1/10
49/49 [==============================] - 57s 1s/step - loss: 0.8022 - accuracy: 0.5722 - val_loss: 0.5891 - val_accuracy: 0.6968
Epoch 2/10
49/49 [==============================] - 51s 1s/step - loss: 0.6088 - accuracy: 0.6889 - val_loss: 0.6040 - val_accuracy: 0.6367
Epoch 3/10
49/49 [==============================] - 50s 1s/step - loss: 0.5875 - accuracy: 0.6931 - val_loss: 0.5815 - val_accuracy: 0.6691
Epoch 4/10
49/49 [==============================] - 50s 1s/step - loss: 0.5452 - accuracy: 0.7273 - val_loss: 0.5580 - val_accuracy: 0.6724
Epoch 5/10
49/49 [==============================] - 51s 1s/step - loss: 0.4400 - accuracy: 0.7984 - val_loss: 0.4497 - val_accuracy: 0.7807
Epoch 6/10
49/49 [==============================] - 50s 1s/step - loss: 0.2815 - accuracy: 0.8844 - val_loss: 0.4555 - val_accuracy: 0.7952
Epoch 7/10
49/49 [==============================] - 50s 1s/step - loss: 0.1584 - accuracy: 0.9460 - val_loss: 0.4733 - val_accuracy: 0.8052
Epoch 8/10
49/49 [==